<a href="https://colab.research.google.com/github/tjturnage/lightning/blob/master/EarthNetwork_lightning_placefiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="+4" color="green"><b>Create lightning placefiles from ENTLN data</b></font>  
<font size="+1" color="gray"><i>Feb 15, 2025</i></font>
<hr>

<u><b>Input</u>:</b> <i>a CSV file of ENTLN archived data</i>  
This can have any name but it can't include spaces and it must end in ".csv"

Instructions for obtaining archived lightning datasets are [here](https://get.earthnetworks.com/hubfs/Customer%20Success/Sferic%20Maps%20Lightning%20Archive.pdf). Make sure you select csv format for download!

A fake sample from a csv dataset is below (your version may not have all the quotation marks, but the code should handle that regardless):

```
"time_utc","type","latitude","longitude","peak_current","ic_height"
"2024-08-30T16:18:16.123456789","1","42.123456","-88.123456","2345.0","6543.0"
```

<u><b>Ouput</u>:</b> <i>two placefiles called "cg_place.txt" and "ic_place.txt" </i>  
These represent CG strike locations and IC flashes, respectively. The files can be downloaded (click folder icon at left to reveal them) and loaded in GR2Analyst.
<br>
<br>
<u><b>Suggestion</u>:</b>  
<i>arrange the stacking order in the GR2Analyst Placefile Manager so the cg_place.txt is on top. That way, the CG strikes won't be concealed by the more numerous IC flashes.</i>   
<br>

---
Questions or bugs? Please contact me at thomas.turnage@noaa.gov


In [5]:
# @title Select the number of minutes keep lightning displayed.<br>Then click the arrow button to execute the script.
minutes_to_keep_display = 15 # @param {"type":"slider","min":5,"max":120,"step":5}
mins = int(minutes_to_keep_display)

from glob import glob
import pandas as pd
from datetime import datetime, timedelta

"""
Code snippet example:
TimeRange: 2025-02-12T17:45:00Z 2025-02-12T17:50:00Z
Object: 40.15885,-89.33891
  Threshold: 400
  Color: 255 255 255

 Icon: 0,0,250,1,1
 End:
"""

"""
csv file data example:
"time_utc","type","latitude","longitude","peak_current","ic_height"
"2024-08-30T16:18:16.210708549","1","37.794473","-90.346058","2688.0","6467.0"
"""

header = """
RefreshSeconds: 30
Color: 255 200 255
IconFile: 1, 16, 16, 8, 8, "https://raw.githubusercontent.com/tjturnage/cloud-radar-server/refs/heads/main/assets/iconfiles/ltg.png"
Font: 1, 11, 1, "Arial"
Font: 2, 14, 1, "Arial"
"""

ltg_csv_file = glob('*.csv')[0]
# Load the CSV file into a DataFrame
df = pd.read_csv(ltg_csv_file, parse_dates=['time_utc'])
df['formatted_time'] = df['time_utc'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df['time_with_timedelta'] = df['time_utc'] + pd.Timedelta(minutes=mins)
df['formatted_time_with_delta'] = df['time_with_timedelta'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
# Convert the remaining columns to appropriate data types
df['type'] = df['type'].astype(int)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['peak_current'] = df['peak_current'].astype(float)
df['ic_height'] = df['ic_height'].astype(float)

icon_dict = {
    'pos1': '1',
    'neg1': '2',
    'pos0': '3',
    'neg0': '4'}

df_ic = df[df['type'] == 1]
df_cg = df[df['type'] == 0]

def write_placefile(df, filename) -> None:
    with open(filename, 'w', encoding='utf-8') as f:
        if filename == 'cg_place.txt':
            f.write("Title: CG Strikes\n")
        else:
            f.write("Title: IC Flashes\n")
        f.write(header)
        for index, row in df.iterrows():
            valid_time = row['formatted_time']
            valid_delta = row['formatted_time_with_delta']
            ltg_type = row['type']  # 1 = intercloud, 0 = cloud to ground
            current = row['peak_current']
            if current < 0:
                sign = 'neg'
            else:
                sign = 'pos'
            f.write(f"\nTimeRange: {valid_time} {valid_delta}\n")
            f.write(f"Object: {row['latitude']},{row['longitude']}\n")
            icon_key = f"{sign}{ltg_type}"
            icon_num = icon_dict[icon_key]
            f.write(f"Icon: 0,0,0,1,{icon_num}\n")
            f.write(f"  Threshold: 400\n")
            f.write(f"End:\n")

write_placefile(df_ic, 'ic_place.txt')
write_placefile(df_cg, 'cg_place.txt')
